In [ ]:
import os
os.getcwd()

: 

In [ ]:
os.chdir('../')

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [ ]:
from deepclassifier.utils import *
from deepclassifier.constants import *

In [ ]:
import os


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks

        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(config.root_dir), Path(config.tensorboard_root_log_dir), Path(model_checkpoint_dir)])

        prepare_callbacks_config = PrepareCallbacksConfig(root_dir=Path(config.root_dir),
                                                           tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                                                           checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
                                                            )
        
        return prepare_callbacks_config

In [ ]:
import tensorflow as tf
import time


class PrepareCallbacks():
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d:%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath,
                                                  save_best_only=True)

    # tb: tensorboard, ckpt: checkpoint
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
            

In [ ]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e